In [218]:
# from google.colab import drive
# drive.mount('/content/drive')

In [219]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [220]:
import warnings
warnings.filterwarnings('ignore')
%config Completer.use_jedi = False # if autocompletion doesnot work in kaggle notebook | hit tab

In [221]:
df_full = pd.read_csv('/content/drive/MyDrive/NLP - Project/April 22/public_data.csv')

df_full

,id,message,label
0,5396,I made a purchase recently but I have decided ...,request_refund
1,2202,"Cancel my transfer, please.",cancel_transfer
2,3768,How do I change my last name?,edit_personal_details
3,4967,A withdraw I tried to make at the ATM was decl...,declined_cash_withdrawal
4,5622,I tried to buy something online yesterday and ...,declined_transfer
...,...,...,...
10461,1982,My top-up hasn't gone through,pending_top_up
10462,2535,Why are you charging me fees for using my card?,card_payment_fee_charged
10463,1424,"The transactions showing up are strange, I thi...",compromised_card
10464,987,Where can I pay with my debit or credit card?,card_acceptance


#### here we are doing some text preprocessing 



In [222]:
df_cleaned = df_full.copy()

In [223]:
# changing the data type to the category to encode into codes 
df_cleaned['Intent'] = df_cleaned.label.astype('category')


In [224]:
df_cleaned.Intent

0                  request_refund
1                 cancel_transfer
2           edit_personal_details
3        declined_cash_withdrawal
4               declined_transfer
                   ...           
10461              pending_top_up
10462    card_payment_fee_charged
10463            compromised_card
10464             card_acceptance
10465               exchange_rate
Name: Intent, Length: 10466, dtype: category
Categories (77, object): ['Refund_not_showing_up', 'activate_my_card', 'age_limit',
                          'apple_pay_or_google_pay', ..., 'visa_or_mastercard', 'why_verify_identity',
                          'wrong_amount_of_cash_received', 'wrong_exchange_rate_for_cash_withdrawal']

In [225]:
df_cleaned.Intent.cat.codes

0        52
1         9
2        31
3        27
4        28
         ..
10461    48
10462    16
10463    23
10464    11
10465    33
Length: 10466, dtype: int8

In [226]:
df_cleaned['Intent']  =  df_cleaned.Intent.cat.codes
df_cleaned.Intent

0        52
1         9
2        31
3        27
4        28
         ..
10461    48
10462    16
10463    23
10464    11
10465    33
Name: Intent, Length: 10466, dtype: int8

In [227]:
from sklearn.model_selection import train_test_split
data_train,data_test = train_test_split(df_cleaned, test_size = 0.3, random_state = 42, stratify = df_cleaned.Intent)

In [228]:
data_train.shape

(7326, 4)

In [229]:
data_train.head(1)

,id,message,label,Intent
7227,8498,I don't think that my top-up worked.,top_up_failed,59


In [230]:
data_test.head(1)

,id,message,label,Intent
2924,6941,I really need to know how to change my pin.,change_pin,22


In [231]:
data_test.shape

(3140, 4)

In [232]:
from tensorflow.keras.utils import to_categorical

In [233]:
to_categorical(data_train.Intent)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

### so far we have cleaned our text data now we need to encode our output in some labels , 
here we have two method to encode 

## Now lets load the model 

In [234]:
!pip install transformers

In [235]:
from transformers import AutoTokenizer,TFBertModel
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert = TFBertModel.from_pretrained('bert-base-cased')


Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [236]:
tokenizer.save_pretrained('bert-tokenizer')
bert.save_pretrained('bert-model')
# for saving model locally and we can load it later on 

In [237]:
### we can use distilbert its lighter cheaper and similar performance 

from transformers import BertTokenizer, TFBertModel, BertConfig,TFDistilBertModel,DistilBertTokenizer,DistilBertConfig
dbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')


Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_layer_norm', 'vocab_projector', 'vocab_transform', 'activation_13']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [238]:
tokenizer('hello this me shivang')

{'input_ids': [101, 19082, 1142, 1143, 188, 22268, 2118, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [239]:
# Tokenize the input (takes some time) 
# here tokenizer using from bert-base-cased
x_train = tokenizer(
    text=data_train.message.tolist(),
    add_special_tokens=True,
    max_length=28,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)


x_test = tokenizer(
    text=data_test.message.tolist(),
    add_special_tokens=True,
    max_length=28,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)



In [240]:
x_test['input_ids']

<tf.Tensor: shape=(3140, 28), dtype=int32, numpy=
array([[  101,   146,  1541, ...,     0,     0,     0],
       [  101,   146,   112, ...,   102,     0,     0],
       [  101,  2353,  1499, ...,     0,     0,     0],
       ...,
       [  101,  1731,  1263, ...,     0,     0,     0],
       [  101,  8667,   106, ...,  3908,  9942,   102],
       [  101,  1109, 14915, ...,     0,     0,     0]], dtype=int32)>

In [241]:
x_train['input_ids']

<tf.Tensor: shape=(7326, 28), dtype=int32, numpy=
array([[  101,   146,  1274, ...,     0,     0,     0],
       [  101,  2009,  1108, ...,     0,     0,     0],
       [  101,  2825,   146, ...,     0,     0,     0],
       ...,
       [  101,  2009,  1110, ...,     0,     0,     0],
       [  101,  2009,  1225, ...,     0,     0,     0],
       [  101,  5718,  1128, ..., 12017,  1103,   102]], dtype=int32)>

In [242]:
x_train

{'input_ids': <tf.Tensor: shape=(7326, 28), dtype=int32, numpy=
array([[  101,   146,  1274, ...,     0,     0,     0],
       [  101,  2009,  1108, ...,     0,     0,     0],
       [  101,  2825,   146, ...,     0,     0,     0],
       ...,
       [  101,  2009,  1110, ...,     0,     0,     0],
       [  101,  2009,  1225, ...,     0,     0,     0],
       [  101,  5718,  1128, ..., 12017,  1103,   102]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(7326, 28), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 1]], dtype=int32)>}

In [243]:
print(data_train.Intent.shape)
print(data_test.Intent.shape)

(7326,)
(3140,)


In [244]:
data_train.Intent

7227    59
9595    20
1244    73
6032    17
7962    10
        ..
7982    75
3027     4
58      47
7859    20
6288     0
Name: Intent, Length: 7326, dtype: int8

### loading some libraries 

In [245]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical

In [246]:
import tensorflow as tf
tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [247]:
max_len = 28
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense

input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
# embeddings = dbert_model(input_ids,attention_mask = input_mask)[0]


embeddings = bert(input_ids,attention_mask = input_mask)[0] #(0 is the last hidden states,1 means pooler_output)
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32,activation = 'relu')(out)

y = Dense(77,activation = 'softmax')(out)
    
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True
# for training bert our lr must be so small

In [248]:
optimizer = Adam(
    learning_rate=5e-05, # this learning rate is for bert model , taken from huggingface website 
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)

# Set loss and metrics
loss =CategoricalCrossentropy(from_logits = True)
metric = CategoricalAccuracy('balanced_accuracy'),
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

In [249]:
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 28)]         0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 28)]         0           []                               
                                                                                                  
 tf_bert_model_4 (TFBertModel)  TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 28,                                          

In [250]:
tf.config.experimental_run_functions_eagerly(False)
tf.config.run_functions_eagerly(False)


#### model fitting and then evaluation

In [251]:
m = to_categorical(data_train.Intent)
m[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [252]:
n = to_categorical(data_test.Intent)
n[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [253]:
train_history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = m,
    validation_data = (
    {'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']}, n
    ),
  epochs=10,
    batch_size=64
)

Epoch 1/10
115/115 [==============================] - 62s 308ms/step - loss: 3.8179 - balanced_accuracy: 0.1275 - val_loss: 2.8163 - val_balanced_accuracy: 0.3818
Epoch 2/10
115/115 [==============================] - 32s 277ms/step - loss: 2.3191 - balanced_accuracy: 0.5034 - val_loss: 1.6504 - val_balanced_accuracy: 0.7102
Epoch 3/10
115/115 [==============================] - 32s 276ms/step - loss: 1.4526 - balanced_accuracy: 0.7364 - val_loss: 1.1774 - val_balanced_accuracy: 0.8035
Epoch 4/10
115/115 [==============================] - 32s 276ms/step - loss: 1.0003 - balanced_accuracy: 0.8363 - val_loss: 0.9381 - val_balanced_accuracy: 0.8452
Epoch 5/10
115/115 [==============================] - 32s 276ms/step - loss: 0.7671 - balanced_accuracy: 0.8766 - val_loss: 0.8191 - val_balanced_accuracy: 0.8605
Epoch 6/10
115/115 [==============================] - 32s 280ms/step - loss: 0.6121 - balanced_accuracy: 0.9077 - val_loss: 0.7491 - val_balanced_accuracy: 0.8732
Epoch 7/10
115/115 [==

lets create a new model and then load the weights 

### Prediction Part

In [254]:
predicted_raw = model.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})

In [255]:
predicted_raw[0]

array([8.7978238e-05, 8.7173341e-04, 9.2683171e-05, 3.4499494e-04,
       1.5152494e-03, 2.3226454e-05, 2.7457133e-06, 3.0462455e-05,
       7.6476994e-05, 1.5018518e-04, 8.5577060e-04, 5.7273905e-04,
       4.7967628e-06, 4.1331559e-06, 2.4132892e-06, 2.3836774e-06,
       1.6977426e-04, 3.5919533e-05, 8.3734139e-06, 5.3711340e-04,
       1.9173326e-04, 3.6071087e-05, 9.7760278e-01, 8.8072346e-05,
       1.4731320e-06, 7.7338382e-06, 6.0984064e-05, 5.1774987e-04,
       3.4294277e-05, 4.5949630e-05, 7.7910026e-06, 1.4608019e-03,
       3.5474202e-06, 1.5789080e-04, 4.1925104e-04, 4.5176537e-05,
       1.3191280e-05, 5.2120267e-05, 1.3062029e-05, 2.0875174e-03,
       6.1293901e-04, 5.8564520e-04, 2.1300202e-03, 2.5010237e-04,
       1.3878386e-03, 1.3991485e-04, 1.1145944e-04, 4.1705653e-06,
       1.9850022e-06, 1.7137518e-03, 6.6706263e-05, 3.7876675e-06,
       2.2395671e-04, 3.2026551e-04, 4.7111124e-04, 1.8051127e-05,
       7.3144381e-04, 1.1395804e-05, 2.0311732e-05, 7.9076388e

In [256]:
y_predicted = np.argmax(predicted_raw, axis = 1)

In [257]:
data_test.Intent

2924    22
451     26
755     48
6806    76
4508    64
        ..
1860    75
5835    71
2928     6
4040    16
1773     1
Name: Intent, Length: 3140, dtype: int8

In [258]:
from sklearn.metrics import classification_report


In [259]:
print(classification_report(data_test.Intent, y_predicted))

              precision    recall  f1-score   support

           0       0.82      0.96      0.89        49
           1       0.90      0.90      0.90        48
           2       0.97      1.00      0.99        36
           3       0.98      1.00      0.99        40
           4       0.97      0.90      0.93        31
           5       0.97      0.88      0.92        40
           6       0.79      0.75      0.77        51
           7       0.93      0.98      0.95        53
           8       0.95      0.81      0.87        47
           9       0.96      0.96      0.96        47
          10       0.90      0.95      0.93        40
          11       0.95      0.75      0.84        24
          12       0.88      0.93      0.91        46
          13       0.92      0.89      0.90        37
          14       0.95      0.88      0.92        43
          15       0.84      0.86      0.85        37
          16       0.86      0.93      0.89        54
          17       0.80    